In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas openpyxl
import pandas as pd
import numpy as np

# Data Preparation

In [ ]:
file_path = "cover-letter-dataset.parquet"

# Kolom yang penting
kolom_utama = ['Job Title', 'Cover Letter']

print("Memuat data...")
df = pd.read_parquet(file_path)

column_names = [
    'Job Title', 'Preferred Qualifications', 'Hiring Company', 'Applicant Name',
    'Past Working Experience', 'Current Working Experience', 'Skillsets',
    'Qualifications', 'Cover Letter'
]
if len(df.columns) == len(column_names):
    df.columns = column_names

print(f"Jumlah baris awal: {len(df)}")

Memuat data...
Jumlah baris awal: 813


In [ ]:
# Kolom yang penting
kolom_utama = ['Job Title', 'Cover Letter']

# a. Normalisasi semua teks
print("Membersihkan teks (lowercase, hapus spasi & baris baru)...")
for col in df.select_dtypes(include=['object']).columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.strip()
        df[col] = df[col].str.replace('\n', ' ', regex=False)

# b. Hapus baris jika kolom Job Title atau Cover Letter kosong
print("Membuang baris dengan data 'Job Title' atau 'Cover Letter' yang kosong...")
rows_before = len(df)
# Ganti string kosong '' dengan NaN (Not a Number) agar bisa di-drop
df[kolom_utama] = df[kolom_utama].replace('', np.nan)
df.dropna(subset=kolom_utama, inplace=True)
rows_after = len(df)
print(f"-> {rows_before - rows_after} baris tidak lengkap dihapus.")

# c. Hapus duplikat berdasarkan kombinasi Job Title dan Cover Letter
print("Membuang duplikat berdasarkan kombinasi 'Job Title' & 'Cover Letter'...")
rows_before = len(df)
df.drop_duplicates(subset=kolom_utama, keep='first', inplace=True)
rows_after = len(df)
print(f"-> {rows_before - rows_after} baris duplikat dihapus.")

Membersihkan teks (lowercase, hapus spasi & baris baru)...
Membuang baris dengan data 'Job Title' atau 'Cover Letter' yang kosong...
-> 0 baris tidak lengkap dihapus.
Membuang duplikat berdasarkan kombinasi 'Job Title' & 'Cover Letter'...
-> 54 baris duplikat dihapus.


In [ ]:
# Mengubah ke format string yang rapi untuk dimasukkan ke dalam prompt
unique_job_titles = df['Job Title'].unique().tolist()
job_title_options = ", ".join([f"'{title}'" for title in unique_job_titles])

print("Dataset bersih berhasil dimuat.")
print(f"Ditemukan {len(unique_job_titles)} Job Title unik.")
print(f"Pilihan Job Title yang akan diberikan ke model:")
job_title_options

Dataset bersih berhasil dimuat.
Ditemukan 51 Job Title unik.
Pilihan Job Title yang akan diberikan ke model:


"'senior java developer', 'data scientist', 'senior data scientist', 'senior web developer', 'ai engineer', 'machine learning engineer', 'data engineer', 'analytics lead', 'senior database administrator', 'intermediate applied scientist', 'system administrator', 'senior machine learning engineer', 'senior data scientist  nlp', 'senior support engineer', 'network engineer', 'generative ai engineer', 'manager, data science and ml engineering', 'senior manager, supply chain analytics', 'mobile app developer', 'ai software developer', 'entry level data scientist', 'software developer', 'senior data scientist in the product research and development (pr&d) group', 'digital analytics consultant', 'lead data scientist', 'risk analytics tech lead', 'machine learning scientist', 'full stack developer', 'cloud engineer', 'senior software engineer', 'python engineer', 'data science/bioinformatics specialist', 'head of people analytics', 'data analyst', 'senior data analyst', 'machine learning deve

# Model IBM Granite

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

# Classification Job Title

In [ ]:
model_parameters = {
    "temperature": 0.01,
    "top_k": 1,
    "top_p": 1.0,
    "max_tokens": 50
}

unique_job_titles = df['Job Title'].unique().tolist()
job_title_options = ", ".join([f"'{title}'" for title in unique_job_titles])

prompt_template = """
Anda adalah seorang asisten HR yang sangat teliti dan ahli dalam memfilter surat lamaran kerja.
Tugas Anda adalah membaca surat lamaran (Cover Letter) di bawah ini dan menentukan posisi pekerjaan (Job Title) mana yang paling sesuai untuk pelamar tersebut.

Pilihlah HANYA SATU nama jabatan dari daftar berikut:
{job_title_options}

Berikut adalah contoh untuk memandu Anda:
Contoh Surat Lamaran: "With over five years in object-oriented programming, particularly Java and Spring Boot, I have successfully developed and deployed scalable microservices that have improved system efficiency by 20%. I am eager to bring my backend development skills to a new challenge."
Contoh Jawaban: 'senior java developer'

Sekarang, analisis surat lamaran di bawah ini.

Surat Lamaran:
"{cover_letter}"

JAWAB HANYA dengan nama jabatannya saja dari daftar pilihan di atas, tanpa penjelasan atau format tambahan.
"""

# --- PROSES KLASIFIKASI ---
# Diambil 3 sampel acak dari data untuk demonstrasi

sample_data = df.sample(n=3, random_state=42)

print("--- Memulai Proses Klasifikasi untuk 3 Sampel Acak ---")

for index, row in sample_data.iterrows():
    cover_letter_text = row['Cover Letter']
    actual_job_title = row['Job Title']

    # Mengisi template dengan data spesifik dari baris ini
    final_prompt = prompt_template.format(
        job_title_options=job_title_options,
        cover_letter=cover_letter_text
    )

    # Memanggil model
    print(f"\n----- Menganalisis Baris ke-{index} -----")
    print(f"Job Title Sebenarnya: {actual_job_title}")

    # Invoke the model
    predicted_job_title = output.invoke(final_prompt).strip().lower().strip("'")

    print(f"Prediksi Model: {predicted_job_title}")

    # Cek status
    if predicted_job_title == actual_job_title:
        print("Status: ✅ TEPAT")
    else:
        print("Status: ❌ KURANG TEPAT")

print("\n--- Proses Selesai ---")

--- Memulai Proses Klasifikasi untuk 3 Sampel Acak ---

----- Menganalisis Baris ke-410 -----
Job Title Sebenarnya: data scientist
Prediksi Model: data scientist
Status: ✅ TEPAT

----- Menganalisis Baris ke-542 -----
Job Title Sebenarnya: network engineer
Prediksi Model: network engineer
Status: ✅ TEPAT

----- Menganalisis Baris ke-101 -----
Job Title Sebenarnya: system administrator
Prediksi Model: system administrator
Status: ✅ TEPAT

--- Proses Selesai ---


# Summarization Cover Letter

In [ ]:
summary_model_parameters = {
    "temperature": 0.5,
    "top_p": 0.9,
    "max_tokens": 256,
}

summary_prompt_template = """
Anda adalah seorang Asisten Perekrutan (Recruitment Assistant) yang cerdas dan efisien.
Tugas Anda adalah membaca Surat Lamaran (Cover Letter) di bawah ini dan membuat ringkasan terstruktur untuk membantu manajer perekrutan memahami profil kandidat dengan cepat.

Gunakan format berikut, dan buat setiap poin ringkas dan informatif:
- Ringkasan Pengalaman: [Sebutkan jumlah tahun pengalaman dan fokus utama kandidat]
- Skillset Utama: [Sebutkan 3-5 skill, bahasa pemrograman, atau teknologi kunci yang disebutkan]
- Posisi Relevan Sebelumnya: [Sebutkan 1-2 posisi atau perusahaan yang ditonjolkan dalam surat]

Jika salah satu informasi di atas tidak dapat ditemukan dalam teks, tulis "Tidak disebutkan".

Berikut adalah contoh yang baik untuk memandu Anda:
Contoh Surat Lamaran: "As a data professional with over 6 years of experience transforming raw data into actionable insights, I am excited to apply for this role. My expertise lies in Python, specifically with libraries like Pandas and Scikit-learn, and I have a proven track record of building predictive models at my previous role at XYZ Corporation. I am adept at SQL for data querying and Tableau for visualization."
Contoh Jawaban:
- Ringkasan Pengalaman: 6+ tahun pengalaman dalam mengubah data menjadi insight.
- Skillset Utama: Python (Pandas, Scikit-learn), SQL, Tableau, Predictive Modeling.
- Posisi Relevan Sebelumnya: Data Professional di XYZ Corporation.

Sekarang, buat ringkasan untuk surat lamaran di bawah ini.

Surat Lamaran:
"{cover_letter}"
"""

# --- PROSES RINGKASAN ---
sample_data_summary = df.sample(n=5, random_state=101)

print("--- Memulai Proses Ringkasan untuk 5 Sampel Acak ---")

for index, row in sample_data_summary.iterrows():
    cover_letter_text = row['Cover Letter']
    job_title = row['Job Title']

    # Mengisi template dengan cover letter dari baris ini
    final_prompt = summary_prompt_template.format(cover_letter=cover_letter_text)

    # Memanggil model
    print(f"\n===== Meringkas untuk Posisi: {job_title.upper()} =====")
    print("\n--- Cover Letter Asli ---")
    print(cover_letter_text)

    # Invoke the model
    summary_response = output.invoke(final_prompt)

    print("\n--- Ringkasan oleh Model Granite ---")
    print(summary_response)

print("\n--- Proses Selesai ---")

--- Memulai Proses Ringkasan untuk 5 Sampel Acak ---

===== Meringkas untuk Posisi: DATA SCIENTIST =====

--- Cover Letter Asli ---
dear hiring manager,  i am writing to express my interest in the data scientist position at xyz biotech. with my strong background in data analysis and experience in the pharmaceutical industry, i believe i would be a valuable asset to your team.  in my current role as a research scientist at xyz biotech, i have been responsible for developing data analysis strategies and presenting insights based on my analysis. i have also worked extensively with genetic and proteomic data, optimizing data sets for ease of access and traceability. my experience in the biotech industry has given me a deep understanding of the importance of data science in improving research and workflows.  i have a passion for science and medicine, and i thrive in fast-paced environments. my strong organizational and time management skills allow me to effectively handle multiple projects 